In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import lasagne
from lasagne.layers import InputLayer, DenseLayer, DropoutLayer
from lasagne.layers.dnn import Conv2DDNNLayer as ConvLayer
from lasagne.layers.dnn import MaxPool2DDNNLayer as PoolLayer
from lasagne.layers import LocalResponseNormalization2DLayer as NormLayer
from lasagne.utils import floatX

Using gpu device 0: GeForce GTX 670 (CNMeM is disabled)


In [3]:
import theano.tensor as T

In [4]:
input_var = T.tensor4('inputs')
target_var = T.ivector('targets')

In [5]:
net = {}
net['input'] = InputLayer((None, 3, 200, 200),input_var=input_var)
net['conv1'] = ConvLayer(net['input'], num_filters=64, filter_size=5, stride=1,
                        nonlinearity = lasagne.nonlinearities.rectify)
#net['norm1'] = NormLayer(net['conv1'], alpha=0.0001) # caffe has alpha = alpha * pool_size
net['pool1'] = PoolLayer(net['conv1'], pool_size=5, stride=5)
net['conv2'] = ConvLayer(net['pool1'], num_filters=64, filter_size=5,
                        nonlinearity = lasagne.nonlinearities.rectify)
net['pool2'] = PoolLayer(net['conv2'], pool_size=5, stride=5)
#net['conv3'] = ConvLayer(net['pool2'], num_filters=128, filter_size=3, pad=1)
#net['conv4'] = ConvLayer(net['conv3'], num_filters=128, filter_size=3, pad=1)
#net['conv5'] = ConvLayer(net['conv4'], num_filters=128, filter_size=3, pad=1)
#net['pool5'] = PoolLayer(net['conv5'], pool_size=3, stride=3, ignore_border=False)
net['fc6'] = DenseLayer(net['pool2'], num_units=512,
                        nonlinearity = lasagne.nonlinearities.rectify)
net['drop6'] = DropoutLayer(net['fc6'], p=0.5)
net['fc7'] = DenseLayer(net['drop6'], num_units=256,
                        nonlinearity = lasagne.nonlinearities.rectify)
net['drop7'] = DropoutLayer(net['fc7'], p=0.5)
net['fc8'] = DenseLayer(net['drop7'], num_units=64,
                        nonlinearity = lasagne.nonlinearities.rectify)
net['drop8'] = DropoutLayer(net['fc8'], p=0.5)
net['fc9'] = DenseLayer(net['drop8'], num_units=2, nonlinearity=lasagne.nonlinearities.softmax)
output_layer = net['fc9']

In [6]:
import matplotlib.image as mpimg

In [7]:
labels =  pd.read_csv('./b37d3960-6909-472b-9ce1-c33b07dbdb66.csv', index_col='id')

In [8]:
labels = labels.sort_index()

In [9]:
data = np.zeros((3969, 3, 200, 200), dtype='float32')

In [10]:
labels.index

Int64Index([   1,    2,    3,    4,    5,    6,    8,    9,   12,   14,
            ...
            4947, 4949, 4950, 4951, 4952, 4953, 4957, 4958, 4959, 4960],
           dtype='int64', name=u'id', length=3969)

In [11]:
k = 0
for i in labels.index:
    with open('./bee_images/train/'+str(i)+'.jpg', 'r') as f:
        img = mpimg.imread(f, format='jpg')
        img2 = np.zeros((3,200,200))
        for j in range(3):
            img2[j,:,:] = img[:,:,j] / np.float(256)
        data[k] = img2.copy()
    k += 1

In [12]:
def add_rev(data):
    dnew = np.zeros(data.shape, dtype='float32')
    k = 0
    for im in data:
        dnew[k] = im[:,:,::-1].copy()
        k += 1
    #print im[:,:,::-1]
    return dnew

def add_ud(data):
    dnew = np.zeros(data.shape, dtype='float32')
    k = 0
    for im in data:
        dnew[k] = im[:,::-1,:].copy()
        k += 1
    return dnew

def add_rud(data):
    dnew = np.zeros(data.shape, dtype='float32')
    k = 0
    for im in data:
        dnew[k] = im[:,::-1,::-1].copy()
        k += 1
    return dnew

In [13]:
X_train, X_val = data[:3000], data[3000:]

In [14]:
X_train = np.concatenate((X_train, add_rev(X_train)))

In [15]:
y_train, y_val = labels.genus.values[:3000].astype('int32'), labels.genus.values[3000:].astype('int32')

In [16]:
y_train = np.concatenate((y_train, y_train))

In [13]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [14]:
import theano

In [15]:
prediction = lasagne.layers.get_output(output_layer)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()
# We could add some weight decay as well here, see lasagne.regularization.

# Create update expressions for training, i.e., how to modify the
# parameters at each training step. Here, we'll use Stochastic Gradient
# Descent (SGD) with Nesterov momentum, but Lasagne offers plenty more.
params = lasagne.layers.get_all_params(output_layer, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.002, momentum=0.9)

# Create a loss expression for validation/testing. The crucial difference
# here is that we do a deterministic forward pass through the network,
# disabling dropout layers.
test_prediction = lasagne.layers.get_output(output_layer, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                        target_var)
test_loss = test_loss.mean()
# As a bonus, also create an expression for the classification accuracy:
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                  dtype=theano.config.floatX)


# Compile a function performing a training step on a mini-batch (by giving
# the updates dictionary) and returning the corresponding training loss:
train_fn = theano.function([input_var, target_var], loss, updates=updates)

# Compile a second function computing the validation loss and accuracy:
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

test_fn = theano.function([input_var], test_prediction)

In [16]:
import time

In [21]:
num_epochs = 1000
batchsize = 50

# Finally, launch the training loop.
print("Starting training...")
# We iterate over epochs:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batchsize, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, 1, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

Starting training...
Epoch 1 of 1000 took 29.062s
  training loss:		0.540663
  validation loss:		0.535070
  validation accuracy:		77.50 %
Epoch 2 of 1000 took 29.045s
  training loss:		0.523094
  validation loss:		0.532241
  validation accuracy:		77.50 %
Epoch 3 of 1000 took 29.025s
  training loss:		0.512196
  validation loss:		0.528031
  validation accuracy:		77.50 %
Epoch 4 of 1000 took 29.062s
  training loss:		0.504657
  validation loss:		0.527730
  validation accuracy:		77.50 %
Epoch 5 of 1000 took 29.018s
  training loss:		0.500190
  validation loss:		0.518361
  validation accuracy:		77.50 %
Epoch 6 of 1000 took 29.098s
  training loss:		0.491219
  validation loss:		0.512244
  validation accuracy:		77.50 %
Epoch 7 of 1000 took 29.003s
  training loss:		0.482430
  validation loss:		0.496166
  validation accuracy:		77.50 %
Epoch 8 of 1000 took 29.136s
  training loss:		0.472475
  validation loss:		0.481338
  validation accuracy:		77.50 %
Epoch 9 of 1000 took 29.169s
  training los

KeyboardInterrupt: 

In [17]:
X_train = data
y_train = labels.genus.values.astype('int32')

In [18]:
X_train = np.concatenate((X_train, add_rev(X_train)))

In [19]:
y_train = np.concatenate((y_train, y_train))

In [20]:
tst = pd.read_csv('./SubmissionFormat.csv')

In [22]:
test_data = np.zeros((992, 3, 200, 200), dtype='float32')

In [23]:
k = 0
for i in tst.id.values:
    with open('./bee_images/test/'+str(i)+'.jpg', 'r') as f:
        img = mpimg.imread(f, format='jpg')
        img2 = np.zeros((3,200,200))
        for j in range(3):
            img2[j,:,:] = img[:,:,j] / np.float(256)
        test_data[k] = img2.copy()
    k += 1

In [21]:
num_epochs = 30

# Finally, launch the training loop.
print("Starting training...")
# We iterate over epochs:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, 50, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))

Starting training...
Epoch 1 of 30 took 36.797s
  training loss:		0.541783
Epoch 2 of 30 took 35.248s
  training loss:		0.524894
Epoch 3 of 30 took 35.223s
  training loss:		0.512466
Epoch 4 of 30 took 34.992s
  training loss:		0.506625
Epoch 5 of 30 took 34.996s
  training loss:		0.494774
Epoch 6 of 30 took 34.945s
  training loss:		0.484159
Epoch 7 of 30 took 34.966s
  training loss:		0.460485
Epoch 8 of 30 took 34.941s
  training loss:		0.434591
Epoch 9 of 30 took 34.870s
  training loss:		0.425599
Epoch 10 of 30 took 34.771s
  training loss:		0.409182
Epoch 11 of 30 took 34.775s
  training loss:		0.393525
Epoch 12 of 30 took 34.816s
  training loss:		0.376791
Epoch 13 of 30 took 34.944s
  training loss:		0.367757
Epoch 14 of 30 took 34.784s
  training loss:		0.356380
Epoch 15 of 30 took 34.756s
  training loss:		0.349472
Epoch 16 of 30 took 34.775s
  training loss:		0.341199
Epoch 17 of 30 took 34.761s
  training loss:		0.330030
Epoch 18 of 30 took 34.802s
  training loss:		0.32612

In [24]:
def iterate_testbatches(inputs, batchsize, shuffle=False):
    assert len(inputs)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt]

In [26]:
ans = []
for t in test_data:
    vals = test_fn(t.reshape((1,3,200,200)))
    vals2 = test_fn(t[:,:,::-1].reshape((1,3,200,200)))
    ans.append((vals[:,1] + vals2[:,1])/2.0)

In [27]:
res = np.array(ans).reshape(992)

In [28]:
tst.genus = res.copy()

In [29]:
tst.to_csv('./fourth.csv', index = False)